In [ ]:
from cryptography.hazmat.primitives.ciphers import Cipher, algorithms, modes
from cryptography.hazmat.backends import default_backend
from cryptography.hazmat.primitives import hashes
from cryptography.hazmat.primitives import padding
from cryptography.hazmat.primitives.kdf.pbkdf2 import PBKDF2HMAC
import os
import base64
import hashlib

# AES Encryption and Decryption Class
class SecureFileShare:
    def __init__(self, password: str):
        """
        Initialize the class with a password for key derivation.
        """
        self.password = password.encode()
        self.salt = os.urandom(16)  # Generate a random salt
        self.key = self.derive_key(self.password, self.salt)
    
    def derive_key(self, password: bytes, salt: bytes) -> bytes:
        """
        Derive a 256-bit key using PBKDF2HMAC with SHA-256.
        """
        kdf = PBKDF2HMAC(
            algorithm=hashes.SHA256(),
            length=32,
            salt=salt,
            iterations=100000,
            backend=default_backend()
        )
        return kdf.derive(password)

    def encrypt_file(self, file_path: str) -> str:
        """
        Encrypt the file using AES GCM mode and return the encrypted data and nonce.
        """
        with open(file_path, 'rb') as f:
            data = f.read()
        
        # Padding data to make it multiple of block size (16 bytes)
        padder = padding.PKCS7(128).padder()
        padded_data = padder.update(data) + padder.finalize()
        
        # AES GCM Mode: AES + Galois/Counter Mode (GCM) for encryption and authentication
        nonce = os.urandom(12)  # Random nonce for GCM mode
        cipher = Cipher(algorithms.AES(self.key), modes.GCM(nonce), backend=default_backend())
        encryptor = cipher.encryptor()
        
        ciphertext = encryptor.update(padded_data) + encryptor.finalize()
        tag = encryptor.tag  # Authentication tag for integrity check

        # Encrypt the data, return the encrypted data, nonce, and tag as base64
        encrypted_data = base64.b64encode(nonce + tag + ciphertext).decode('utf-8')
        return encrypted_data

    def decrypt_file(self, encrypted_data: str, output_path: str) -> None:
        """
        Decrypt the encrypted data and write the original data to output path.
        """
        encrypted_data = base64.b64decode(encrypted_data)
        
        # Extract nonce, tag, and ciphertext from the encrypted data
        nonce = encrypted_data[:12]
        tag = encrypted_data[12:28]
        ciphertext = encrypted_data[28:]
        
        cipher = Cipher(algorithms.AES(self.key), modes.GCM(nonce, tag), backend=default_backend())
        decryptor = cipher.decryptor()
        
        decrypted_data = decryptor.update(ciphertext) + decryptor.finalize()
        
        # Remove padding and write the decrypted data to a file
        unpadder = padding.PKCS7(128).unpadder()
        unpadded_data = unpadder.update(decrypted_data) + unpadder.finalize()
        
        with open(output_path, 'wb') as f:
            f.write(unpadded_data)

    def verify_file_integrity(self, file_path: str) -> str:
        """
        Create a SHA-256 hash of the file to ensure integrity.
        """
        sha256_hash = hashlib.sha256()
        with open(file_path, 'rb') as f:
            for byte_block in iter(lambda: f.read(4096), b""):
                sha256_hash.update(byte_block)
        return sha256_hash.hexdigest()

# Demonstration of Secure File Sharing with Encryption and Integrity Verification
if __name__ == "__main__":
    # Example usage of the SecureFileShare class
    password = "super_secure_password"  # Password for encryption key
    secure_file_share = SecureFileShare(password)
    
    # Encrypt the file
    encrypted_data = secure_file_share.encrypt_file("example_file.txt")
    print("Encrypted File Data (Base64):")
    print(encrypted_data)
    
    # Save the encrypted data to a file (simulating file transfer)
    with open("encrypted_file.txt", 'w') as f:
        f.write(encrypted_data)
    
    # Decrypt the file
    secure_file_share.decrypt_file(encrypted_data, "decrypted_example_file.txt")
    print("Decryption complete, saved to 'decrypted_example_file.txt'.")

    # Verify the file integrity using SHA-256 hash
    original_hash = secure_file_share.verify_file_integrity("example_file.txt")
    print(f"Original File Integrity Hash: {original_hash}")
    
    # Verify after decryption
    decrypted_hash = secure_file_share.verify_file_integrity("decrypted_example_file.txt")
    print(f"Decrypted File Integrity Hash: {decrypted_hash}")
    
    if original_hash == decrypted_hash:
        print("File integrity verified. The file has not been tampered with.")
    else:
        print("File integrity failed. The file may have been tampered with.")